In [10]:
!pip install awscli --upgrade

  Using cached botocore-1.38.21-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.12.0-py3-none-any.whl.metadata (1.7 kB)
Using cached botocore-1.38.21-py3-none-any.whl (13.6 MB)
Using cached s3transfer-0.12.0-py3-none-any.whl (84 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.38
    Uninstalling botocore-1.37.38:
      Successfully uninstalled botocore-1.37.38
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.11.5
    Uninstalling s3transfer-0.11.5:
      Successfully uninstalled s3transfer-0.11.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.38.21 which is incompatible.
boto3 1.37.17 requires botocore<1.38.0,>=1.37.17, but you have botocore 1.38.21 which is incompatible.
boto3 1.37.17 requires s3transfer<0.12.0,>=0.11.0, but you have s3transfer 0.12.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install boto3

  Using cached botocore-1.37.38-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.11.5-py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.5 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/13.5 MB 5.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.6/13.5 MB 4.6 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/13.5 MB 4.4 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/13.5 MB 4.3 MB/s eta 0:00:03
   --------------- ------------------------ 5.2/13.5 MB 4.2 MB/s eta 0:00:02
   ----------------- ---------------------- 6.0/13.5 MB 4.1 MB/s eta 0:00:02
   -------------------- ------------------- 6.8/13.5 MB 4.1 MB/s eta 0:00:02
   ---------------------- ----------------- 7.6/13.5 MB 4.1 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/13.5 MB 4.1 MB/s eta 0:00:02
   ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.37.38 which is incompatible.
awscli 1.40.20 requires botocore==1.38.21, but you have botocore 1.37.38 which is incompatible.
awscli 1.40.20 requires s3transfer<0.13.0,>=0.12.0, but you have s3transfer 0.11.5 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!aws configure


2025-05-22 11:27:51 yt-mlflowbucket


File association not found for extension .py


In [4]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/")

In [16]:
# Set or create an experiment
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2025/05/22 12:37:14 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://yt-mlflowbucket/812669549667746837', creation_time=1747897634252, experiment_id='812669549667746837', last_update_time=1747897634252, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [18]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [19]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/05/22 12:37:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837/runs/daa69ecb8d144da7a543809b38550629
🧪 View experiment at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837


2025/05/22 12:38:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837/runs/521807ea57124c9e8437fc1c3779ec9f
🧪 View experiment at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837


2025/05/22 12:38:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837/runs/06f164283ebb466da7b31615a36678f7
🧪 View experiment at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837


2025/05/22 12:39:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837/runs/3ce81a6fffa640f29236a2d50ed76eb2
🧪 View experiment at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837


2025/05/22 12:39:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837/runs/ff6f8b46aa254fad804055e99f366f63
🧪 View experiment at: http://ec2-13-233-74-14.ap-south-1.compute.amazonaws.com:5000/#/experiments/812669549667746837
